In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import torch
import re
import os
import transformers 
from torch.utils.data import Dataset
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
from huggingface_hub import login


model_name = "gpt2-large"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, is_decoder= True).to(device)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [2]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)


input_prompts = [
    "Write a function to calculate the factorial of a number.",
    "How can you reverse a string in Python?",
    "Create a function to check if a number is a palindrome.",
    "Write a program to find the greatest common divisor (GCD) of two numbers.",
    "How can you implement the Fibonacci sequence in Python?",
    "Write a Python script to read a CSV file and print its contents.",
    "How do you create a new text file and write data into it in Python?",
    "Write a program to count the number of lines in a text file.",
    "How can you append data to an existing file in Python?",
    "Write a script to search for a specific word in a file.",
    "Implement a stack using a Python list.",
    "How do you implement a queue using a Python list?",
    "Write a program to sort a list of integers in ascending order.",
    "How can you merge two sorted lists in Python?",
    "Create a function to find the second largest number in a list.",
    "Write a program to check if two strings are anagrams.",
    "How do you count the occurrences of each element in a list?",
    "Write a Python function to flatten a nested list.",
    "How can you generate all permutations of a string in Python?",
    "Implement a binary search algorithm for a sorted list.",
    "Write a function to find the depth of a binary tree.",
    "How can you implement a linked list in Python?",
    "Write a program to traverse a binary tree in in-order, pre-order, and post-order.",
    "How do you find the shortest path in a graph using Dijkstra's algorithm?",
    "Create a function to validate if a given string has balanced parentheses.",
    "Write a program to find the longest substring without repeating characters.",
    "How can you remove duplicates from a list in Python?",
    "Write a function to calculate the power of a number using recursion.",
    "How do you implement a simple calculator in Python?",
    "Create a function to find all prime numbers up to a given limit."
]

large_dataset = []
for prompt in input_prompts:
    generated_texts = generator(prompt, max_length=1024, num_return_sequences=15, temperature=0.7)
    for gen in generated_texts:
        large_dataset.append(gen['generated_text'])


with open("generated_data.txt", "w", encoding="utf-8") as f:
    for text in large_dataset:
        f.write(text + "\n\n")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [3]:
from sklearn.model_selection import train_test_split

chunk_size = 512
chunks = [text[i:i+chunk_size] for text in large_dataset for i in range(0, len(text), chunk_size)]

train_data, val_data = train_test_split(chunks, test_size = 0.1, random_state = 42)

In [8]:
train_inputs = tokenizer(
    train_data,
    return_tensors = 'pt',
    max_length = chunk_size,
    truncation = True,
    padding = "max_length",
)

val_inputs = tokenizer(
    val_data,
    return_tensors = 'pt',
    max_length = chunk_size,
    truncation = True,
    padding = "max_length",
)


train_inputs['labels'] = train_inputs['input_ids'].clone()
val_inputs['labels'] = val_inputs['input_ids'].clone()

In [9]:
class TextDataset(Dataset):
    def __init__(self, inputs):
        self.input_ids = inputs['input_ids']
        self.attention_mask = inputs['attention_mask']
        self.labels = inputs['labels']


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, idx):
        return {
            "input_ids" : self.input_ids[idx],
            "attention_mask" : self.attention_mask[idx],
            "labels" : self.labels[idx]
        }


train_dataset = TextDataset(train_inputs)
eval_dataset = TextDataset(val_inputs)

In [31]:
new_model_name = 'gpt2'
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(new_model_name)
model = AutoModelForCausalLM.from_pretrained(new_model_name).to(device)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [44]:
training_arguments = TrainingArguments(
    output_dir = "./result",
    overwrite_output_dir= True,
    do_train = True,
    do_eval = True,
    save_strategy = "steps",
    eval_strategy = "steps",
    logging_strategy = "steps",
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 2,
    lr_scheduler_type = "cosine",
    logging_steps = 10,
    warmup_ratio = 0.1,
    save_steps = 100,
    learning_rate = 1e-5,
    num_train_epochs = 8,
    max_steps = 500,
    max_grad_norm = 1.0,
    report_to = "none",
    load_best_model_at_end = True,
    metric_for_best_model = "eval_loss",
    fp16 = True
    
)

In [45]:
from transformers import EarlyStoppingCallback
from transformers import DataCollatorForLanguageModeling
from torch.optim import AdamW

early_stop_callback = EarlyStoppingCallback(early_stopping_patience = 2)
optimizer = AdamW(model.parameters(), lr = 1e-5, weight_decay = 0.1)
data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer,
    mlm = False
)

trainer = Trainer(
    model = model,
    data_collator = data_collator,
    args = training_arguments,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    optimizers = (optimizer, None),
    callbacks = [early_stop_callback]
)

max_steps is given, it will override any value given in num_train_epochs


In [47]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
10,1.267300,1.054682
20,1.200300,1.054211
30,1.192800,1.053403
40,1.164100,1.051566
50,1.203100,1.048601
60,1.246400,1.047036
70,1.115100,1.045370
80,1.215200,1.043112
90,1.234500,1.042529
100,1.130700,1.040197


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=500, training_loss=1.1475318851470948, metrics={'train_runtime': 1306.4529, 'train_samples_per_second': 6.123, 'train_steps_per_second': 0.383, 'total_flos': 2087200751616000.0, 'train_loss': 1.1475318851470948, 'epoch': 2.824858757062147})

In [48]:
model.save_pretrained("./fine_tuning_model_gpt2")
tokenizer.save_pretrained("./fine_tuning_model_gpt2")

('./fine_tuning_model_gpt2/tokenizer_config.json',
 './fine_tuning_model_gpt2/special_tokens_map.json',
 './fine_tuning_model_gpt2/vocab.json',
 './fine_tuning_model_gpt2/merges.txt',
 './fine_tuning_model_gpt2/added_tokens.json',
 './fine_tuning_model_gpt2/tokenizer.json')

In [49]:
from transformers import AutoModelForCausalLM, AutoTokenizer


model = AutoModelForCausalLM.from_pretrained("./fine_tuning_model_gpt2")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuning_model_gpt2")


input_text = "Write a function to calculate the factorial of a number."
input_ids = tokenizer.encode(input_text, return_tensors="pt")


output = model.generate(
    input_ids,
    max_length=64,
    num_return_sequences=3,
    no_repeat_ngram_size=2,
    temperature=0.7,
    top_k=10,
    top_p=0.95,
    do_sample=True,
    early_stopping=True,
    eos_token_id=tokenizer.eos_token_id,
)

for i, out in enumerate(output):
    print("\n")
    generated_text = tokenizer.decode(out, skip_special_tokens=True)
    words = generated_text.split()


    for j in range(0, len(words), 20):
        line = ' '.join(words[j:j + 20])
        print(line)
    print("\n")



/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.




Write a function to calculate the factorial of a number. function fact(n) { var fact = Math.floor(Math.random() * fact.exp(1)); }
function fact2(fact){ var n = fact; var x = n * n; return fact * x; }




Write a function to calculate the factorial of a number. function fact_factorial(a, b) if a == b: return -1 else:
print("Number is " + a + b + "") return 0 else : return 1 .def fact(n): for i in
range(




Write a function to calculate the factorial of a number. function fact(n, n) return fact * n . The fact
function can be used to find the number of elements in a list. For example, if you have a string,
you can find it by using the string.find() function




In [50]:
from transformers import AutoModelForCausalLM, AutoTokenizer


model = AutoModelForCausalLM.from_pretrained("./fine_tuning_model_gpt2")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuning_model_gpt2")


input_text = "Write a function to calculate the factorial of a number."
input_ids = tokenizer.encode(input_text, return_tensors="pt")


output = model.generate(
    input_ids,
    max_length=256,
    num_return_sequences=3,
    no_repeat_ngram_size=2,
    temperature=0.7,
    top_k=10,
    top_p=0.95,
    do_sample=True,
    early_stopping=True,
    eos_token_id=tokenizer.eos_token_id,
)

for i, out in enumerate(output):
    print("\n")
    generated_text = tokenizer.decode(out, skip_special_tokens=True)
    words = generated_text.split()


    for j in range(0, len(words), 20):
        line = ' '.join(words[j:j + 20])
        print(line)
    print("\n")



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




Write a function to calculate the factorial of a number. def fact(a, b): return a.fact(0) .sort(b) # sort by the
number of elements in the list return fact The fact function returns the sum of the elements of all the
numbers in a list. It is a bit more complex than the above example, but it is still a good
starting point. The function is very simple to implement, and it can be implemented in Python as a class. This
is also a great way to learn about factoring. In fact, the function calculates the square root of 2.0. So,
it's not so much that it takes the same number as it does a fact number, because the two are
not the equal. But it gives you an idea of how factors work. For example: #!/usr/bin/env python import sys import
time def fact2(n): for i in range(1000): # find the first element in n of n if i == 0:
# return the smallest element of ids ix = ices[i] if is(i+1): x = x + ichor(x, ics(1)) else: y




Write a function to calculate the factorial of a number. def fact_factorial(n):